In [11]:
import os
from PIL import Image
from tqdm import tqdm
import splitfolders

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_facenet import FaceNet
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import json

In [12]:
input_dir = "Cropped_50"
output_dir = "Cropped_50_Resized_160"

In [13]:
def resize_with_padding(input_dir, output_dir, target_size=(160, 160)):
    os.makedirs(output_dir, exist_ok=True)

    for root, _, files in os.walk(input_dir):
        for file in tqdm(files, desc=f"Processing images in {root}"):
            try:
                input_path = os.path.join(root, file)
                output_subdir = os.path.join(output_dir, os.path.relpath(root, input_dir))
                os.makedirs(output_subdir, exist_ok=True)

                output_path = os.path.join(output_subdir, file)

                with Image.open(input_path) as img:
                    img = img.convert("RGB")

                    if img.width < target_size[0] and img.height < target_size[1]:
                        scale_factor = min(
                            target_size[0] / img.width,
                            target_size[1] / img.height
                        )
                        new_width = int(img.width * scale_factor)
                        new_height = int(img.height * scale_factor)
                        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

                    else:
                        img.thumbnail(target_size, Image.Resampling.LANCZOS)

                    new_img = Image.new("RGB", target_size, (0, 0, 0))

                    offset_x = (target_size[0] - img.width) // 2
                    offset_y = (target_size[1] - img.height) // 2
                    new_img.paste(img, (offset_x, offset_y))

                    new_img.save(output_path)

            except Exception as e:
                print(f"Error processing file {file}: {e}")

In [14]:
resize_with_padding(input_dir, output_dir)

Processing images in Cropped_50: 0it [00:00, ?it/s]
Processing images in Cropped_50\Youssef: 100%|██████████| 54/54 [00:00<00:00, 134.72it/s]


In [15]:
dataset_dir = "Cropped_50_Resized_160"
output_dir = "train_val_split"

splitfolders.ratio(dataset_dir, output=output_dir, seed=42, ratio=(0.8, 0.2), group_prefix=None)

print("Dataset split into training and validation sets.")

Copying files: 749 files [00:00, 1153.49 files/s]

Dataset split into training and validation sets.


In [16]:
train_dir = "train_val_split/train"
val_dir = "train_val_split/val"

train_datagen = ImageDataGenerator(rescale=1.0/255, rotation_range=20, zoom_range=0.2)
val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(160, 160), batch_size=32, class_mode='categorical'
)
val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=(160, 160), batch_size=32, class_mode='categorical'
)

Found 596 images belonging to 14 classes.
Found 153 images belonging to 14 classes.


In [29]:
facenet = FaceNet()

base_model = facenet.model

num_classes = len(train_generator.class_indices)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

fine_tuned_model = Model(inputs=base_model.input, outputs=output)

fine_tuned_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [30]:
fine_tuned_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator),
    callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    ]
)

Epoch 1/10
19/19 [==============================] - 20s 650ms/step - loss: 2.4347 - accuracy: 0.6174 - val_loss: 2.1281 - val_accuracy: 0.9673 - lr: 0.0010
Epoch 2/10
19/19 [==============================] - 10s 522ms/step - loss: 1.8754 - accuracy: 0.9228 - val_loss: 1.5023 - val_accuracy: 0.9739 - lr: 0.0010
Epoch 3/10
19/19 [==============================] - 11s 579ms/step - loss: 1.2743 - accuracy: 0.9295 - val_loss: 0.9091 - val_accuracy: 0.9804 - lr: 0.0010
Epoch 4/10
19/19 [==============================] - 9s 469ms/step - loss: 0.7999 - accuracy: 0.9480 - val_loss: 0.5248 - val_accuracy: 0.9869 - lr: 0.0010
Epoch 5/10
19/19 [==============================] - 10s 527ms/step - loss: 0.5512 - accuracy: 0.9396 - val_loss: 0.3333 - val_accuracy: 0.9869 - lr: 0.0010
Epoch 6/10
19/19 [==============================] - 10s 546ms/step - loss: 0.3951 - accuracy: 0.9513 - val_loss: 0.2397 - val_accuracy: 0.9869 - lr: 0.0010
Epoch 7/10
19/19 [==============================] - 11s 559ms/ste

In [33]:
fine_tuned_model.save_weights('fine_tuned_model_weights.h5')